In [2]:
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import re
import time
import ast

### (1) Lista de professores em excel
#### Colunas:
1. Name
2. Institute
3. GAA	
4. Function	(Efetivo/Substituto)
5. from (formato: yys)
6. to (formato: yys)

Colunas 5 e 6 necessárias apenas quando professor trocou de instituto dentro da universidade. Caso contrário, usa-se 0 e 999, respectivamente.


In [3]:
arquivo = 'tabela_professores_ICT.xlsx'
lista_docentes = pd.read_excel(arquivo).values

### (2) Conectando ao SIGAA
1. Preencher suário e senha para logar no SIGAA usando uma janela controlada pelo seleniun
2. Usando Chrome para acessar a web:
    a. Você precisa do ChromeDriver 
    b. Baixar uma versão compatível com seu OS e com seu Chrome (https://chromedriver.chromium.org/downloads)
    c. Descompactar e colocar na mesma pasta que este arquivo. Abrir o executável
3. Executar as duas células abaixo

In [ ]:
user = 'cpf'   # usuário (cpf)
pasw = '***'   # senha

In [ ]:

driver = webdriver.Chrome()
# Open the website
driver.get('https://sigaa.unifei.edu.br/sigaa/verTelaLogin.do')

# Select the id box
id_box = driver.find_element_by_name('user.login')
# Send id information
id_box.send_keys(user)

# Find password box
pass_box = driver.find_element_by_name('user.senha')
# Send password
pass_box.send_keys(pasw)

# Find login button
login_button = driver.find_element_by_xpath("//input[@value='Entrar']")
# Click login
login_button.click()

### (3) Página de buscas
1. IR Ensino > Consultas > Turmas
2. Na página que abrir, uncheck TODAS as opções de busca.
3. Executar as células abaixo.

In [8]:
# IR Ensino > Consultas > Turmas
# Des-select TUDO

tabela_turmas = []
tabela_disciplinas = []

for docente in lista_docentes:
    docente_box = driver.find_element_by_name('form:inputNomeDocente')
    docente_box.clear()
    docente_box.send_keys(docente[0])

    search_button = driver.find_element_by_name('form:buttonBuscar')
    search_button.click()
    
    time.sleep(1)
    
    table = driver.find_element_by_id('lista-turmas').find_elements_by_xpath('tbody/tr/td')

    table_list = []
    for row in table: table_list.append(row.text)
    
    for n, line in enumerate(table_list):
        if line.find('GRADUAÇÃO)') > 0 or line.find('SENSU)') > 0:
            codigo = line.split(' - ')[0]
            
            # Extrai o nome da disciplina mesmo que haja termos entre parênteses 
            nome_disciplina = ' ('.join(' - '.join(line.split(' - ')[1:]).split(' (')[0:-1])
            
            tipo = line.split('(')[-1][0:-1]
            
            contador = 0
            
        elif re.search('20[0-9][0-9].[1-2]', line):
            anoperiodo = line
        
            turma = table_list[n+1].split(' ')[1]
            
            horario = table_list[n+6].split(' (')[0]
        
            local = table_list[n+7]
        
            matriculados = table_list[n+8].split('/')[0]
            
            lista_bruta = re.split('[(][0-9]+[h][)][,]*[ ]*[e]*', table_list[n+2])
            lista_nomes = []
            for lin in lista_bruta:
                if lin != '': lista_nomes.append(' '.join(lin.split()))
                    
            for nome in lista_nomes:
                cargahoraria = table_list[n+2].split(nome + ' (')[1].split('h)')[0]
            
                anoper = int(anoperiodo[2:4])*10 + int(anoperiodo[-1])
                
                contador = contador + 1
    
                if anoper >= int(docente[4]) and anoper <= int(docente[5]):
                    tabela_turmas.append([codigo, anoperiodo, turma, nome, cargahoraria, horario, local, matriculados])
                    if contador == 1:
                        tabela_disciplinas.append([codigo, nome_disciplina, tipo])

### (4) Tratando os dados
1. Retira duplicatas
2. Transforma em DataFrames de Pandas
3. Classifica as disciplinas em teórcas ou práticas
4. Salva dados em excel e em csv
5. Exemplo de leitura dos arquivos csv

In [9]:
tabela_turmas.sort()
tabela_turmas_unq = []
for i, z in enumerate(tabela_turmas):
    if i == 0: tabela_turmas_unq.append(z)
    elif z != tabela_turmas[i-1]: tabela_turmas_unq.append(z)

tabela_disciplinas.sort()
tabela_disciplinas_unq = []
for i, z in enumerate(tabela_disciplinas):
    if i == 0: tabela_disciplinas_unq.append(z)
    elif z != tabela_disciplinas[i-1]: tabela_disciplinas_unq.append(z)

In [10]:
for lin in tabela_turmas_unq:
    print(' * '.join(lin))

BAC001.1 * 2011.1 * 24988 * CARLOS HENRIQUE DA SILVEIRA * 13 *  *  * 46
BAC001.1 * 2011.1 * 24988 * FLAVIO FONTENELLE LOQUE * 15 *  *  * 46
BAC001.1 * 2011.1 * 24988 * LILIAN BARROS PEREIRA * 4 *  *  * 46
BAC001.1 * 2011.1 * 24992 * LILIAN BARROS PEREIRA * 10 *  *  * 40
BAC001.1 * 2011.1 * 24992 * MILTON JOSE ZAMBONI * 12 *  *  * 40
BAC001.1 * 2011.1 * 24992 * ROGER JUNIO CAMPOS * 10 *  *  * 40
BAC001.2 * 2011.1 * 24973 * CARLOS HENRIQUE DA SILVEIRA * 12 *  *  * 46
BAC001.2 * 2011.1 * 24973 * LILIAN BARROS PEREIRA * 4 *  *  * 46
BAC001.2 * 2011.1 * 24975 * JOSE EUGENIO LOPES DE ALMEIDA * 8 *  *  * 45
BAC001.2 * 2011.1 * 24975 * LILIAN BARROS PEREIRA * 8 *  *  * 45
BAC001.2 * 2011.1 * 24977 * LILIAN BARROS PEREIRA * 6 *  *  * 40
BAC001.2 * 2011.1 * 24977 * MILTON JOSE ZAMBONI * 4 *  *  * 40
BAC001.2 * 2011.1 * 24977 * ROGER JUNIO CAMPOS * 6 *  *  * 40
BAC003 * 2010.1 * 25039 * ALESSANDRO AUGUSTO NUNES CAMPOS * 48 *  *  * 27
BAC003 * 2010.1 * 25039 * JOSE CARLOS DE LACERDA * 6 *  *  * 27

ECA023.2 * 2016.2 * 02 * FERNANDA RODRIGUES DA SILVA * 8 * 2M45 * Prédio 2 - Sala 2211 * 19
ECA023.2 * 2016.2 * 02 * JOÃO LUCAS DA SILVA * 8 * 2M45 * Prédio 2 - Sala 2211 * 19
ECA023.2 * 2016.2 * 03 * JOÃO LUCAS DA SILVA * 8 * 4T12 * Prédio 2 - Sala 2211 * 17
ECA023.2 * 2016.2 * 03 * ROSIMEIRE APARECIDA JERONIMO * 8 * 4T12 * Prédio 2 - Sala 2211 * 17
ECA023.2 * 2016.2 * 04 * JOÃO LUCAS DA SILVA * 8 * 4T12 * Prédio 2 - Sala 2211 * 16
ECA023.2 * 2016.2 * 04 * ROSIMEIRE APARECIDA JERONIMO * 8 * 4T12 * Prédio 2 - Sala 2211 * 16
ECA023.2 * 2017.2 * 01 * JOÃO LUCAS DA SILVA * 16 * 2M45 * Prédio 2 - Sala 2211 * 15
ECA023.2 * 2017.2 * 02 * JOÃO LUCAS DA SILVA * 16 * 2M45 * Prédio 2 - Sala 2211 * 16
ECA023.2 * 2017.2 * 03 * JOÃO LUCAS DA SILVA * 16 * 2M23 * Prédio 2 - Sala 2211 * 8
ECA023.2 * 2017.2 * 04 * JOÃO LUCAS DA SILVA * 16 * 2M23 * Prédio 2 - Sala 2211 * 4
ECA025.1 * 2014.1 * 27100 * DAIR JOSE DE OLIVEIRA * 32 * 56N12 * S12P * 11
ECA025.1 * 2014.1 * 27100 * DENIS DE CARVALHO BRAGA * 32 

ECO037 * 2018.1 * 02 * WANDRE NUNES DE PINHO VELOSO * 48 * 7M345 * Prédio 1 - Linf1 - Sala 1209 * 16
ECO038 * 2013.2 * 27322 * PAULO JOSE LAGE ALVARENGA * 32 * 4N12 * C02 * 18
ECO038 * 2014.2 * 27323 * PAULO JOSE LAGE ALVARENGA * 32 * 4N12 * C02 * 6
ECO038 * 2016.2 * 01 * PAULO JOSE LAGE ALVARENGA * 32 * 6T34 * Prédio 2 - Sala 2207 * 11
ECO044.1 * 2015.1 * 27326 * WALTER AOIAMA NAGAI * 16 * 4M4 * Linf1p * 20
ECO044.1 * 2015.2 * 27327 * WALTER AOIAMA NAGAI * 16 * 4T1 * Linf1p * 18
ECO044.1 * 2016.1 * 01 * WALTER AOIAMA NAGAI * 16 * 4M2 * Prédio 2 - Sala 2208 * 20
ECO044.1 * 2016.2 * 01 * WALTER AOIAMA NAGAI * 16 * 5M2 * Prédio 2 - Sala 2208 * 4
ECO044.1 * 2017.1 * 01 * WALTER AOIAMA NAGAI * 16 * 5T1 * Prédio 2 - Sala 2208 * 14
ECO044.1 * 2017.2 * 01 * WALTER AOIAMA NAGAI * 16 * 2M2 * Prédio 2 - Sala 2208 * 23
ECO044.1 * 2018.1 * 01 * WALTER AOIAMA NAGAI * 16 * 2M2 * Prédio 2 - Sala 2208 * 9
ECO044.1 * 2019.1 * 01 * ERICK OLIVEIRA RODRIGUES * 16 * 3T5 * Prédio 2 - Sala 2207 * 9
ECO044.2 

EEL027.2 * 2013.2 * 27508 * FREDERICO FERREIRA VIANA MATOS * 32 * 5T12 * LMMep * 8
EEL027.2 * 2013.2 * 27515 * FREDERICO FERREIRA VIANA MATOS * 32 * 5N12 * LMMep * 2
EEL027.2 * 2013.2 * 27526 * FREDERICO FERREIRA VIANA MATOS * 32 * 6M45 * LMMep * 6
EEL027.2 * 2014.1 * 27509 * FREDERICO FERREIRA VIANA MATOS * 32 * 2M23 * LMMep * 9
EEL027.2 * 2014.1 * 27516 * FREDERICO FERREIRA VIANA MATOS * 32 * 2M45 * LMMep * 9
EEL027.2 * 2014.1 * 27522 * FREDERICO FERREIRA VIANA MATOS * 32 * 2T12 * LMMep * 10
EEL027.2 * 2014.1 * 27527 * FREDERICO FERREIRA VIANA MATOS * 32 * 2T34 * LMMep * 9
EEL027.2 * 2015.1 * 27510 * FREDERICO FERREIRA VIANA MATOS * 32 * 4M23 * LMMep * 10
EEL027.2 * 2015.1 * 27517 * FREDERICO FERREIRA VIANA MATOS * 32 * 4M45 * LMMep * 8
EEL027.2 * 2015.1 * 27523 * FREDERICO FERREIRA VIANA MATOS * 32 * 5M23 * LMMep * 11
EEL027.2 * 2015.1 * 27528 * EBEN EZER PRATES DA SILVEIRA * 32 * 5M45 * LMMep * 10
EEL027.2 * 2016.1 * 01 * IVAN PAULO DE FARIA * 32 * 3T5 3N1 * LME (Anexo 2 - Sala 15)

EELI15 * 2019.2 * 04 * DIOGO LEONARDO FERREIRA DA SILVA * 32 * 6M23 * Prédio 2 - Sala 2314 * 9
EELI16 * 2017.1 * 01 * IVAN PAULO DE FARIA * 64 * 24M45 * Prédio 2 - Sala 2302 * 30
EELI16 * 2017.2 * 01 * IVAN PAULO DE FARIA * 64 * 24M45 * Anexo 3 - Sala 16 * 29
EELI16 * 2018.1 * 01 * IVAN PAULO DE FARIA * 64 * 24M23 * Anexo 3 - Sala 6 * 56
EELI16 * 2018.2 * 01 * IVAN PAULO DE FARIA * 64 * 24M45 * Anexo 3 - Sala 14 * 43
EELI16 * 2019.1 * 01 * IVAN PAULO DE FARIA * 64 * 35M23 * Prédio 2 - Sala 2316 * 43
EELI16 * 2019.2 * 01 * IVAN PAULO DE FARIA * 64 * 35M23 * Prédio 2 (3ª: Sala 2307 e 5ª: Sala 2317) * 40
EELI17 * 2017.2 * 01 * EBEN EZER PRATES DA SILVEIRA * 64 * 24M45 * Anexo 3 - Sala 17 * 43
EELI17 * 2018.1 * 01 * GEOVANE LUCIANO DOS REIS * 64 * 46T12 * Anexo 3 - Sala 10 * 22
EELI17 * 2018.2 * 01 * GEOVANE LUCIANO DOS REIS * 64 * 24M45 * Anexo 3 - Sala 3 * 47
EELI17 * 2019.1 * 01 * EBEN EZER PRATES DA SILVEIRA * 64 * 35M23 * Anexo 3 - Sala 12 * 39
EELI17 * 2019.2 * 01 * EBEN EZER PRATES 

ELT020.2 * 2013.1 * 27969 * CLODUALDO VENICIO DE SOUSA * 32 * 5T12 * LEp * 7
ELT020.2 * 2013.1 * 27970 * CLODUALDO VENICIO DE SOUSA * 32 * 5T34 * LEp * 1
ELT021.1 * 2010.1 * 27973 * MARCEL FERNANDO DA COSTA PARENTONI * 64 *  *  * 10
ELT021.1 * 2010.1 * 27973 * WELINTON LA FONTAINE LOPES * 64 *  *  * 10
ELT021.2 * 2010.1 * 27972 * MARCEL FERNANDO DA COSTA PARENTONI * 16 *  *  * 10
ELT021.2 * 2010.1 * 27972 * WELINTON LA FONTAINE LOPES * 16 *  *  * 10
ELT022.1 * 2010.1 * 27980 * ALESSANDRO AUGUSTO NUNES CAMPOS * 32 *  *  * 20
ELT022.1 * 2010.1 * 27980 * DAVIDSON LAFITTE FIRMO * 48 *  *  * 20
ELT022.1 * 2010.2 * 27981 * ALUISIO ALVES DE MELO BENTO * 80 *  *  * 18
ELT022.2 * 2010.1 * 27974 * DAVIDSON LAFITTE FIRMO * 16 *  *  * 7
ELT022.2 * 2010.1 * 27974 * IVAN LUCAS ARANTES * 16 *  *  * 7
ELT022.2 * 2010.1 * 27976 * DAVIDSON LAFITTE FIRMO * 16 *  *  * 3
ELT022.2 * 2010.1 * 27976 * IVAN LUCAS ARANTES * 16 *  *  * 3
ELT022.2 * 2010.1 * 27978 * ALESSANDRO AUGUSTO NUNES CAMPOS * 16 *  *  * 6


In [11]:
for lin in tabela_disciplinas_unq:
    print(' * '.join(lin))

BAC001.1 * Ciência, Tecnologia e Sociedade * GRADUAÇÃO
BAC001.2 * Ciência, Tecnologia e Sociedade (Prática) * GRADUAÇÃO
BAC003 * DESENHO APLICADO * GRADUAÇÃO
BAC003.2 * DESENHO APLICADO (PRÁTICA) * GRADUAÇÃO
BAC004.1 * INFORMÁTICA * GRADUAÇÃO
BAC004.2 * INFORMÁTICA (PRÁTICA) * GRADUAÇÃO
BAC005 * MATEMÁTICA I * GRADUAÇÃO
BAC006.1 * ELETRICIDADE * GRADUAÇÃO
BAC006.2 * ELETRICIDADE (PRÁTICA) * GRADUAÇÃO
BAC025.1 * METODOLOGIA DA PESQUISA CIENTÍFICA * GRADUAÇÃO
BAC025.2 * METODOLOGIA DA PESQUISA CIENTÍFICA (PRÁTICA) * GRADUAÇÃO
BACI01.1 * CIÊNCIA, TECNOLOGIA E SOCIEDADE * GRADUAÇÃO
BACI01.2 * CIÊNCIA, TECNOLOGIA E SOCIEDADE (PRÁTICA) * GRADUAÇÃO
CCF130.1 * Algoritmos e Estrutura de Dados III * GRADUAÇÃO
CCF130.2 * Algoritmos e Estrutura de Dados III (Prática) * GRADUAÇÃO
CCF902 * Tópicos Especiais em Programação II * GRADUAÇÃO
CCO003 * LINGUAGENS DE PROGRAMAÇÃO * GRADUAÇÃO
CCO012 * Fundamentos de Programação II * GRADUAÇÃO
CCO013 * FUNDAMENTOS DE PROGRAMAÇÃO * GRADUAÇÃO
CCO017 * Programaçã

In [12]:
tabela_turmas_unq_df = pd.DataFrame(tabela_turmas_unq)
tabela_turmas_unq_df.columns = ['Code', 'Year-Period', 'Class', 'Name', 'Hours', 'Day-Time', 'Place', "No Students"]
tabela_turmas_unq_df.head()

,Code,Year-Period,Class,Name,Hours,Day-Time,Place,No Students
0,BAC001.1,2011.1,24988,CARLOS HENRIQUE DA SILVEIRA,13,,,46
1,BAC001.1,2011.1,24988,FLAVIO FONTENELLE LOQUE,15,,,46
2,BAC001.1,2011.1,24988,LILIAN BARROS PEREIRA,4,,,46
3,BAC001.1,2011.1,24992,LILIAN BARROS PEREIRA,10,,,40
4,BAC001.1,2011.1,24992,MILTON JOSE ZAMBONI,12,,,40


In [13]:
tabela_turmas_unq_df.to_excel(r'tabela_turmas_unq_192_df.xlsx', index = None, header=True)
tabela_turmas_unq_df.to_csv(r'tabela_turmas_unq_192_df.csv', index = None, header=True)

In [14]:
tabela_turmas_horarios = []
for turma in tabela_turmas_unq:
    for piece in turma[5].split():
        turno = re.findall('[MTN]', piece)[0]
        [dias, horas] = re.split('[MTN]', piece)
        for d in list(dias):
            for h in list(horas):
                tabela_turmas_horarios.append([turma[0], turma[1], turma[2], turma[3], turma[4], d, turno+h])
tabela_turmas_horarios_df = pd.DataFrame(tabela_turmas_horarios)
tabela_turmas_horarios_df.columns = ['Code', 'Year-Period', 'Class', 'Name', 'Hours', 'WeekDay', 'DayTime']
tabela_turmas_horarios_df.head()

,Code,Year-Period,Class,Name,Hours,WeekDay,DayTime
0,BAC003.2,2013.1,25014,RICARDO SHITSUKA,24,2,T3
1,BAC003.2,2013.1,25014,RICARDO SHITSUKA,24,2,T4
2,BAC003.2,2013.1,25014,RICARDO SHITSUKA,24,2,T5
3,BAC003.2,2013.1,25014,RODRIGO APARECIDO DA SILVA BRAGA,24,2,T3
4,BAC003.2,2013.1,25014,RODRIGO APARECIDO DA SILVA BRAGA,24,2,T4


In [15]:
tabela_turmas_horarios_df.to_excel(r'tabela_turmas_horarios_192_df.xlsx', index = None, header=True)
tabela_turmas_horarios_df.to_csv(r'tabela_turmas_horarios_192_df.csv', index = None, header=True)

In [16]:
for dis in tabela_disciplinas_unq:
    if dis[0].find('.2') >= 0 or dis[1].lower().find('labor') >= 0:
        tipo = 'Lab'
    else: 
        tipo = 'Theory'
    dis.append(tipo)

tabela_disciplinas_unq_df = pd.DataFrame(tabela_disciplinas_unq)
tabela_disciplinas_unq_df.columns = ['Code', 'Course-Name', 'Level', 'Type']
tabela_disciplinas_unq_df.head()

,Code,Course-Name,Level,Type
0,BAC001.1,"Ciência, Tecnologia e Sociedade",GRADUAÇÃO,Theory
1,BAC001.2,"Ciência, Tecnologia e Sociedade (Prática)",GRADUAÇÃO,Lab
2,BAC003,DESENHO APLICADO,GRADUAÇÃO,Theory
3,BAC003.2,DESENHO APLICADO (PRÁTICA),GRADUAÇÃO,Lab
4,BAC004.1,INFORMÁTICA,GRADUAÇÃO,Theory


In [17]:
tabela_disciplinas_unq_df.to_excel(r'tabela_disciplinas_unq_192_df.xlsx', index = None, header=True)
tabela_disciplinas_unq_df.to_csv(r'tabela_disciplinas_unq_192_df.csv', index = None, header=True)

In [18]:
with open('tabela_turmas_unq_list.txt', 'w') as f:
    for line in tabela_turmas_unq:
        f.write("%s\n" % line)

In [19]:
with open('tabela_disciplinas_unq_list.txt', 'w') as f:
    for line in tabela_disciplinas_unq:
        f.write("%s\n" % line)

In [20]:
lista_exemplo = []
with open('tabela_disciplinas_unq_list.txt', 'r') as g:
    for line in g:
        lista_exemplo.append(ast.literal_eval(line))
lista_exemplo[0][1]

'Ciência, Tecnologia e Sociedade'